In [1]:
from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet
import lightgbm as lg
from sklearn.svm import LinearSVC
import pandas as pd 
import numpy as np
import pickle
import os
from sklearn.model_selection import GridSearchCV as  gv
import optuna
import shap
import matplotlib.pyplot as plt 

c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### INSERTING DATA

In [2]:
data_dir="C:/Users/Arda/Downloads/Upwork/Portfolio/Betting_Asistant/data/"
data=pd.read_csv("{}datamart.csv".format(data_dir))

C:\Users\Arda\AppData\Local\Temp\ipykernel_15072\3281985020.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("{}datamart.csv".format(data_dir))


In [3]:
data= data[data["fixture.status.short"]=="FT"].reset_index(drop=True)

In [4]:
data[data["league.season"]==2024].groupby(["league.name","league.country"])["fixture.id"].count()

league.name          league.country
2. Bundesliga        Germany           364
A-League             Australia         199
Allsvenskan          Sweden            400
Bundesliga           Austria           132
                     Germany           362
Championship         England           740
Eerste Divisie       Netherlands       456
Ekstraklasa          Poland            310
Eliteserien          Norway            400
Eredivisie           Netherlands       358
HNL                  Croatia           176
Indian Super League  India             247
J1 League            Japan             660
Jupiler Pro League   Belgium           376
K League 1           South-Korea        90
La Liga              Spain             274
League One           England           776
League Two           England           664
Ligue 1              France            314
Ligue 2              France            360
Paulista - A1        Brazil            112
Premier League       England           280
                  

In [5]:
feature_data=data.iloc[:,28:]
feature_data

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
0,1,4.0,5.0,9.0,3.0,2.5,0.500,1.0,0.0,493.0,...,0,0,1,0,0,0,1,0,0,1.0
1,0,5.0,3.0,11.5,4.5,2.0,0.420,2.0,0.0,338.0,...,0,0,1,0,0,0,1,0,0,1.0
2,1,2.0,5.0,8.0,3.0,1.5,0.440,1.5,0.0,416.0,...,0,0,1,0,0,0,1,0,0,4.0
3,0,5.0,4.5,11.0,5.0,1.0,0.425,2.0,0.0,340.5,...,0,1,0,0,0,0,0,1,0,0.0
4,1,2.5,6.0,11.0,4.5,1.5,0.345,1.5,0.0,261.5,...,0,0,1,0,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1,2.0,3.5,16.5,4.0,1.5,0.395,2.0,0.0,306.5,...,0,0,1,0,0,0,0,0,1,1.0
132757,0,5.5,5.5,11.0,4.5,1.0,0.545,1.0,0.0,417.0,...,0,1,0,0,0,0,0,0,1,2.0
132758,1,4.5,6.5,10.5,6.5,1.0,0.580,2.5,0.0,491.5,...,0,1,0,0,0,1,0,0,0,1.0
132759,1,4.5,6.0,12.5,6.0,4.0,0.595,3.5,0.0,472.5,...,0,0,0,1,0,0,0,1,0,2.0


In [6]:
#We will normalize feature which's range over 1

In [7]:

feature_data_v1=pd.concat([data[["fixture.id","league.id","league.season","team_id"]],
                            feature_data],axis=1)


In [8]:
#Distrubition of goals are skewed to rigth
#In order to ballance distrubition with out damaging cause of the project
# We will assing 6 to over 6 goals

#### Train Test 

In [9]:
fixtures=feature_data_v1["fixture.id"].unique()
fix_size= len(fixtures)
fix_size

66569

In [10]:
train= feature_data_v1[data["fixture_date"]<="2025-01-31"]

test=feature_data_v1[data["fixture_date"]>"2025-01-31"]
test

,fixture.id,league.id,league.season,team_id,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
123417,1202742,210,2024,1475,1,5.0,6.5,14.0,4.0,2.0,...,0,0,0,1,0,0,0,0,1,1.0
123418,1202742,210,2024,1018,0,2.5,5.5,14.5,2.0,0.5,...,0,1,0,0,0,0,0,1,0,1.0
123419,1202743,210,2024,1017,1,5.5,2.0,14.0,5.5,2.0,...,0,1,0,0,0,0,0,0,1,3.0
123420,1202743,210,2024,608,0,3.5,7.0,11.5,5.0,1.0,...,0,1,0,0,0,0,0,0,1,2.0
123421,1202744,210,2024,1483,1,3.5,4.5,10.5,3.5,1.0,...,1,0,0,0,1,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1327719,207,2024,1013,1,2.0,3.5,16.5,4.0,1.5,...,0,0,1,0,0,0,0,0,1,1.0
132757,1327720,207,2024,565,0,5.5,5.5,11.0,4.5,1.0,...,0,1,0,0,0,0,0,0,1,2.0
132758,1327720,207,2024,551,1,4.5,6.5,10.5,6.5,1.0,...,0,1,0,0,0,1,0,0,0,1.0
132759,1327721,207,2024,606,1,4.5,6.0,12.5,6.0,4.0,...,0,0,0,1,0,0,0,1,0,2.0


In [11]:
lmodel=lg.LGBMRegressor()

In [12]:
input_cols=train.columns[4:-1]
input_cols

Index(['Home', 'Shots_on_Goal', 'Shots_off_Goal', 'Foul', 'Corner_Kicks',
       'Offsides', 'Ball_Possession_Rate', 'Yellow_Cards', 'Red_Cards',
       'Total_Passes',
       ...
       'Rank_Name_last_2_3_Oppose', 'Rank_Name_title_Oppose',
       'Rise_Need_1_point_needed_Oppose', 'Rise_Need_3_point_for_catch_Oppose',
       'Rise_Need_3_point_for_chase_Oppose', 'Rise_Need_top_Oppose',
       'Fall_Prevent_1_point_for_run_Oppose',
       'Fall_Prevent_3_point_for_run_Oppose', 'Fall_Prevent_bottom_Oppose',
       'Fall_Prevent_comfort_Oppose'],
      dtype='object', length=113)

In [13]:
otrain,otest=train_test_split(train,test_size=0.3,random_state=42)

In [14]:
target= "Goal"

In [15]:
models= {"lg":lg.LGBMRegressor,
         "ela":ElasticNet,
         "lr":LinearRegression
         }

In [16]:


def try_model(trymod):
    
    
        
    trymod.fit(y=otrain[target], X=otrain[list(input_cols)])
    pred=trymod.predict(otest[list(input_cols)])
    pred[pred<0]=0
   

    return np.mean(abs(pred-otest[target]).sum())



Models

In [17]:

def model_obj(name):
    def objective(trial):

        if name=="lg":
            result=try_model(models["lg"]( 
                boosting_type=trial.suggest_categorical("boosting_type",
                                                        ["gbdt","dart"]),

                num_leaves= trial.suggest_int("num_leaves",
                                            3,50),
                max_depth=trial.suggest_int("max_depth",
                                            -1,20),
                learning_rate= trial.suggest_float("learning_rate",
                                                0.001,0.1),
                reg_alpha= trial.suggest_float("reg_alpha",
                                            0.0,0.1),

                reg_lambda= trial.suggest_float("reg_lambda",
                                                0.0,0.1),
                verbosity=-1
            ))

            return result      
        
        elif name=="ela":
            result=try_model(models["ela"]( 
                            alpha= trial.suggest_float("alpha",
                                                       0.5,5),
                            l1_ratio=trial.suggest_float("l1_ratio",
                                                         0.1,1),
                            fit_intercept=False

            
                            ) )
        
            return result
        
        
        
    return objective
        
        

In [18]:
list(models.keys())

['lg', 'ela', 'lr']

In [19]:
saves={}
for name in list(models.keys())[:-1]:
    
    study = optuna.create_study(direction="minimize")
    
    study.optimize(model_obj(name)
               , n_trials=30)
    
    saves[name]= study.best_params

[I 2025-05-05 16:16:10,757] A new study created in memory with name: no-name-c4d7b5c2-ca21-415f-b03a-30e6f7cb09e4
[I 2025-05-05 16:16:11,761] Trial 0 finished with value: 37232.585303644766 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 7, 'max_depth': -1, 'learning_rate': 0.005165539482016492, 'reg_alpha': 0.030741213215190934, 'reg_lambda': 0.06606810293738291}. Best is trial 0 with value: 37232.585303644766.
[I 2025-05-05 16:16:12,978] Trial 1 finished with value: 34787.96015596152 and parameters: {'boosting_type': 'dart', 'num_leaves': 12, 'max_depth': 18, 'learning_rate': 0.07602066452697406, 'reg_alpha': 0.016791530061618678, 'reg_lambda': 0.05495227979921305}. Best is trial 1 with value: 34787.96015596152.
[I 2025-05-05 16:16:14,561] Trial 2 finished with value: 38332.653519644686 and parameters: {'boosting_type': 'dart', 'num_leaves': 49, 'max_depth': 5, 'learning_rate': 0.004304593679430821, 'reg_alpha': 0.07439264510709662, 'reg_lambda': 0.06672990678911611}. Best is

In [20]:
def fit_model(model,mod_dict,train,test):

    
    model_in=model(**mod_dict)
    model_in.fit(y=train[target], X=train[list(input_cols)])
    pred=model_in.predict(test[list(input_cols)])
    pred[pred<0]=0
    pred[pred>6]=6
   
    return np.mean(abs(pred-test[target])),model_in

In [21]:
def pred_model(model,main_data):
    pred=model.predict(main_data[list(input_cols)])
    pred[pred<0]=0
   
    return pred

In [22]:

lg_score,lg_model=fit_model(models["lg"],saves["lg"],train,test)
lg_preds=pred_model(lg_model,test)

lg_score

np.float64(0.8593128498723508)

In [23]:
ela_score,ela_model=fit_model(models["ela"],saves["ela"],train,test)
ela_preds=pred_model(ela_model,test)
ela_score

np.float64(0.9066990597688839)

In [24]:
ens_pred=(lg_preds+ela_preds)/2


In [25]:
abs(ens_pred-test["Goal"]).sum()/test.shape[0]

np.float64(0.874365364353877)

In [26]:
lr_score,lr_model= fit_model(models["lr"],{},train,test)
lr_preds= pred_model(lr_model,test)
lr_score

np.float64(0.890824063238219)

In [27]:
def combinations(iterable, r):

    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [28]:
pred_dict= {"lr":lr_preds, "ela":ela_preds, "lg":lg_preds}

In [29]:
pkeys=list(pred_dict.keys())

In [30]:
searched=list(iter(combinations(pkeys,2)))+[[i] for i in pkeys]+[pkeys]

In [31]:
wining=[]
best_score=10

for i in  searched:
    final_pred=0
    print(i)
    
    for k in i:
        final_pred+= pred_dict[k]
    
    final_pred=final_pred/len(i)

    score=abs(final_pred-test["Goal"]).sum()/final_pred.shape[0]

    if best_score>score:

        wining=i
        best_score=score

('lr', 'ela')
('lr', 'lg')
('ela', 'lg')
['lr']
['ela']
['lg']
['lr', 'ela', 'lg']


In [32]:
def pred_ensembler(models,pre_dict):
    final_pred=0
    for m in  models:
        final_pred+= pre_dict[m]
    
    return final_pred/len(models)

In [33]:
pred_ensembler(wining,pred_dict)

array([1.2017885 , 1.07404061, 1.14358623, ..., 1.42473107, 1.91590931,
       0.86327914], shape=(2542,))

In [34]:
import pickle as pkl

In [35]:
models.keys()

dict_keys(['lg', 'ela', 'lr'])

In [36]:
fitted_models= {"lg":lg_model, "ela":ela_model, "lr":lr_model}

In [37]:
def model_saver(wining,models):

    best_models={}
    for m in  wining:
        best_models[m]=models[m]
    
    curdir=os.getcwd()[:-9]

    with open(f"{curdir}models\\best_models.pkl","wb") as o:
        
            pkl.dump(best_models,o)

In [38]:
model_saver(wining,fitted_models)

In [39]:
test["pred_ela"]=pred_ensembler(wining,pred_dict)
test["ae"]= test[["pred_ela","Goal"]].apply(lambda x: x["pred_ela"]-x["Goal"] ,axis=1)

C:\Users\Arda\AppData\Local\Temp\ipykernel_15072\2973983054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred_ela"]=pred_ensembler(wining,pred_dict)
C:\Users\Arda\AppData\Local\Temp\ipykernel_15072\2973983054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["ae"]= test[["pred_ela","Goal"]].apply(lambda x: x["pred_ela"]-x["Goal"] ,axis=1)


In [40]:

def up_max(x):
    d=x[x>=0]
    if len(d)>0:
        return d.max()
    else:
        return 0

def down_min(x):
    d=x[x<0]
    if len(d)>0:
        return d.min()
    else:
        return 0

    
    

In [41]:
quantiles=test.groupby(["league.id","team_id",],as_index=False)["ae"].aggregate(["median",up_max,down_min])
quantiles

,league.id,team_id,median,up_max,down_min
0,39,33,-0.023679,1.163617,-1.494490
1,39,34,0.591475,1.023105,-2.791194
2,39,35,0.603000,1.536948,-1.738443
3,39,36,-0.786016,1.345021,-0.998133
4,39,39,-0.263278,0.149055,-0.829964
...,...,...,...,...,...
488,475,7848,0.206927,0.858580,-0.836839
489,475,7865,0.030882,0.938953,-1.882742
490,475,10018,0.823095,1.041231,-0.935314
491,475,10037,-0.158343,0.834698,-1.087905


In [42]:
quantiles.sort_values("down_min")

,league.id,team_id,median,up_max,down_min
18,39,65,-0.865653,1.094555,-5.776400
100,61,91,-1.326625,0.179161,-5.324147
28,40,63,-0.677126,0.702946,-5.176647
366,197,619,-1.866154,0.075846,-5.026270
397,207,565,-0.979701,1.444833,-4.620319
...,...,...,...,...,...
437,235,1080,0.627925,0.627925,0.000000
170,88,203,0.893961,1.041435,0.000000
189,89,411,0.159105,0.230600,0.000000
186,89,204,0.861346,1.367402,0.000000


In [43]:
test

,fixture.id,league.id,league.season,team_id,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,...,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal,pred_ela,ae
123417,1202742,210,2024,1475,1,5.0,6.5,14.0,4.0,2.0,...,0,1,0,0,0,0,1,1.0,1.201789,0.201789
123418,1202742,210,2024,1018,0,2.5,5.5,14.5,2.0,0.5,...,0,0,0,0,0,1,0,1.0,1.074041,0.074041
123419,1202743,210,2024,1017,1,5.5,2.0,14.0,5.5,2.0,...,0,0,0,0,0,0,1,3.0,1.143586,-1.856414
123420,1202743,210,2024,608,0,3.5,7.0,11.5,5.0,1.0,...,0,0,0,0,0,0,1,2.0,1.343838,-0.656162
123421,1202744,210,2024,1483,1,3.5,4.5,10.5,3.5,1.0,...,0,0,1,0,1,0,0,1.0,0.996051,-0.003949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1327719,207,2024,1013,1,2.0,3.5,16.5,4.0,1.5,...,1,0,0,0,0,0,1,1.0,1.028613,0.028613
132757,1327720,207,2024,565,0,5.5,5.5,11.0,4.5,1.0,...,0,0,0,0,0,0,1,2.0,0.978523,-1.021477
132758,1327720,207,2024,551,1,4.5,6.5,10.5,6.5,1.0,...,0,0,0,1,0,0,0,1.0,1.424731,0.424731
132759,1327721,207,2024,606,1,4.5,6.0,12.5,6.0,4.0,...,0,1,0,0,0,1,0,2.0,1.915909,-0.084091


Coming Matches

In [44]:
data_dir="C:/Users/Arda/Downloads/Upwork/Portfolio/Betting_Asistant/data/"
data=pd.read_csv("{}datamart.csv".format(data_dir))
new_data= data[data["fixture.status.short"]!="FT"].reset_index(drop=True)
new_data

C:\Users\Arda\AppData\Local\Temp\ipykernel_15072\1173197005.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("{}datamart.csv".format(data_dir))


,fixture.id,fixture.timezone,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.season,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
0,1202777,UTC,Split,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,0,1,0,0,1,0,0,0,NaN
1,1202777,UTC,Split,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,0,1,0,0,0,0,1,0,NaN
2,1202778,UTC,Koprivnica,Not Started,NS,NaN,210,HNL,Croatia,2024,...,1,0,0,0,1,0,1,0,0,NaN
3,1202778,UTC,Koprivnica,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,1,0,0,0,0,0,0,1,NaN
4,1202779,UTC,Zagreb,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,1,0,0,0,0,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,1362792,UTC,Aalborg,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,0,1,0,1,0,0,0,NaN
480,1362793,UTC,Haderslev,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,1,0,0,0,0,0,0,1,NaN
481,1362793,UTC,Haderslev,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,1,0,0,0,0,0,1,NaN
482,1362794,UTC,Lyngby,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,1,0,0,0,1,0,0,NaN


In [45]:
new_data[input_cols].dropna(axis=0)

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_last_2_3_Oppose,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose
0,0,2.5,5.0,15.0,5.5,1.5,0.520,2.0,0.0,335.5,...,0,0,0,1,0,0,1,0,0,0
1,1,3.5,6.0,13.0,4.0,3.5,0.535,1.5,0.0,379.5,...,0,0,0,1,0,0,0,0,1,0
2,1,3.5,4.5,15.0,3.5,3.5,0.490,2.5,0.0,386.5,...,0,1,0,0,0,1,0,1,0,0
3,0,3.5,7.0,17.0,3.5,2.0,0.520,1.5,0.0,387.0,...,0,0,1,0,0,0,0,0,0,1
4,0,1.5,3.0,13.0,2.0,0.5,0.415,2.5,0.5,353.5,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0,5.5,3.0,11.0,4.0,1.0,0.420,2.0,0.0,405.0,...,0,0,0,0,1,0,1,0,0,0
480,1,3.5,5.0,9.5,3.5,2.0,0.560,0.5,0.0,517.5,...,0,0,1,0,0,0,0,0,0,1
481,0,3.5,2.0,9.5,2.5,1.0,0.410,2.0,0.0,459.5,...,1,0,0,1,0,0,0,0,0,1
482,0,5.5,4.0,15.5,5.0,3.0,0.400,1.0,0.0,373.0,...,1,0,0,1,0,0,0,1,0,0


In [46]:

new_preds=pred_model(lg_model,new_data)
#new_preds=new_preds/3

In [47]:
new_data.columns[20:]

Index(['score.fulltime.away', 'fixture.status.extra', 'league.standings',
       'fixture_date', 'fixture_start_time', 'home_point', 'away_point',
       'team_id', 'Home', 'Shots_on_Goal',
       ...
       'Rank_Name_title_Oppose', 'Rise_Need_1_point_needed_Oppose',
       'Rise_Need_3_point_for_catch_Oppose',
       'Rise_Need_3_point_for_chase_Oppose', 'Rise_Need_top_Oppose',
       'Fall_Prevent_1_point_for_run_Oppose',
       'Fall_Prevent_3_point_for_run_Oppose', 'Fall_Prevent_bottom_Oppose',
       'Fall_Prevent_comfort_Oppose', 'Goal'],
      dtype='object', length=122)

In [48]:
check_data=new_data[["fixture.id","league.id",'league.name', 
          'league.country','league.round',
           'teams.home.name', 'teams.away.name',
           'fixture_date',"team_id","Home"]]
check_data["preds"]=new_preds

C:\Users\Arda\AppData\Local\Temp\ipykernel_15072\214754067.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_data["preds"]=new_preds


In [49]:
check_data_v1=check_data.merge(quantiles)
check_data_v1["Lower_q"]=check_data_v1.preds+check_data_v1["down_min"]
check_data_v1["Upper_q"]= check_data_v1.preds+check_data_v1["up_max"]

In [50]:
up_comming=check_data_v1[check_data_v1.fixture_date<="2025-04-31"]
up_comming_v1=up_comming.groupby(list(up_comming.columns[:7]),as_index=False)[["preds","Lower_q","Upper_q"]].sum()


In [51]:
up_comming_v1[up_comming_v1["league.country"]=="Turkey"]

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q
166,1238101,203,Süper Lig,Turkey,Regular Season - 29,Sivasspor,Adana Demirspor,2.120781,0.163965,4.327380
167,1238102,203,Süper Lig,Turkey,Regular Season - 29,Besiktas,Galatasaray,2.050561,-1.871771,3.554841
168,1238103,203,Süper Lig,Turkey,Regular Season - 29,Antalyaspor,Alanyaspor,2.225437,-1.228820,4.370734
169,1238104,203,Süper Lig,Turkey,Regular Season - 29,Kayserispor,Hatayspor,2.033835,-0.629045,4.041979
170,1238105,203,Süper Lig,Turkey,Regular Season - 29,Konyaspor,Gazişehir Gaziantep,2.372498,-0.517787,4.475356
171,1238106,203,Süper Lig,Turkey,Regular Season - 29,Trabzonspor,Goztepe,2.032291,-0.596412,4.776661
172,1238107,203,Süper Lig,Turkey,Regular Season - 29,Samsunspor,Kasimpasa,2.161253,-2.550188,3.290315
173,1238108,203,Süper Lig,Turkey,Regular Season - 29,Eyüpspor,Istanbul Basaksehir,2.231451,-0.605809,4.534970
174,1238109,203,Süper Lig,Turkey,Regular Season - 29,BB Bodrumspor,Fenerbahce,2.044694,0.351646,4.073750


In [52]:
up_comming_v1.sort_values("preds")

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q
197,1324019,98,J1 League,Japan,Regular Season - 7,Kyoto Sanga,Sanfrecce Hiroshima,0.869781,-0.221395,0.869781
27,1208750,140,La Liga,Spain,Regular Season - 29,Getafe,Villarreal,0.991673,0.991673,1.848063
193,1324015,98,J1 League,Japan,Regular Season - 7,FC Tokyo,Kawasaki Frontale,1.159639,1.159639,2.118687
198,1324020,98,J1 League,Japan,Regular Season - 7,Fagiano Okayama,Yokohama F. Marinos,1.164053,1.164053,2.272623
191,1324013,98,J1 League,Japan,Regular Season - 7,Kashima,Vissel Kobe,1.218581,1.218581,1.752447
...,...,...,...,...,...,...,...,...,...,...
187,1280805,188,A-League,Australia,Regular Season - 24,Melbourne Victory,Adelaide United,2.986738,-1.421966,5.787424
24,1208729,140,La Liga,Spain,Regular Season - 27,Barcelona,Osasuna,3.041806,-1.024296,3.041806
52,1213129,235,Premier League,Russia,Regular Season - 22,Zenit,Rubin,3.048709,3.037099,5.212670
32,1210146,89,Eerste Divisie,Netherlands,Regular Season - 32,Excelsior,Jong Utrecht,3.242821,0.578929,4.837871


In [53]:
up_comming_v1["odd_up"]=(up_comming_v1["Upper_q"]-2.5)/(up_comming_v1["Upper_q"]-up_comming_v1["Lower_q"])
up_comming_v1["odd_down"]=(2.5-up_comming_v1["Lower_q"])/(up_comming_v1["Upper_q"]-up_comming_v1["Lower_q"])

In [54]:
up_comming_v1.sort_values("odd_up",ascending=False).head(10)

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q,odd_up,odd_down
52,1213129,235,Premier League,Russia,Regular Season - 22,Zenit,Rubin,3.048709,3.037099,5.212670,1.246877,-0.246877
55,1213132,235,Premier League,Russia,Regular Season - 22,CSKA Moscow,Makhachkala,2.483679,2.483679,3.549132,0.984681,0.015319
31,1208754,140,La Liga,Spain,Regular Season - 29,Real Madrid,Leganes,2.542099,2.357141,5.068590,0.947313,0.052687
150,1231964,94,Primeira Liga,Portugal,Regular Season - 27,Estrela,Sporting CP,2.417061,2.388588,3.590871,0.907333,0.092667
194,1324016,98,J1 League,Japan,Regular Season - 7,Albirex Niigata,Gamba Osaka,2.320266,2.320266,4.194618,0.904109,0.095891
102,1217250,42,League Two,England,Regular Season - 26,Carlisle,Milton Keynes Dons,2.304159,2.142610,4.974076,0.873779,0.126221
190,1324012,98,J1 League,Japan,Regular Season - 7,Cerezo Osaka,Urawa,2.345603,2.297530,3.403353,0.816906,0.183094
195,1324017,98,J1 League,Japan,Regular Season - 7,Shimizu S-pulse,Shonan Bellmare,2.281692,2.281692,3.323627,0.790479,0.209521
56,1213133,235,Premier League,Russia,Regular Season - 22,Dynamo,FC Orenburg,2.318532,2.318532,3.155154,0.783095,0.216905
41,1210155,89,Eerste Divisie,Netherlands,Regular Season - 32,Jong PSV,Den Bosch,2.309027,2.110801,3.841334,0.775099,0.224901


In [55]:
up_comming_v1["Lower_q"]=up_comming_v1["Lower_q"].apply(lambda x: 0 if x<0 else x)

In [56]:
up_comming_v1.sort_values("odd_down",ascending=False)

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q,odd_up,odd_down
148,1231943,94,Primeira Liga,Portugal,Regular Season - 24,GIL Vicente,Benfica,1.627632,1.064373,1.627632,-1.548787,2.548787
197,1324019,98,J1 League,Japan,Regular Season - 7,Kyoto Sanga,Sanfrecce Hiroshima,0.869781,0.000000,0.869781,-1.494000,2.494000
191,1324013,98,J1 League,Japan,Regular Season - 7,Kashima,Vissel Kobe,1.218581,1.218581,1.752447,-1.400262,2.400262
54,1213131,235,Premier League,Russia,Regular Season - 22,Krylia Sovetov,Lokomotiv,2.335723,2.189499,2.335723,-1.123464,2.123464
27,1208750,140,La Liga,Spain,Regular Season - 29,Getafe,Villarreal,0.991673,0.991673,1.848063,-0.761261,1.761261
...,...,...,...,...,...,...,...,...,...,...,...,...
194,1324016,98,J1 League,Japan,Regular Season - 7,Albirex Niigata,Gamba Osaka,2.320266,2.320266,4.194618,0.904109,0.095891
150,1231964,94,Primeira Liga,Portugal,Regular Season - 27,Estrela,Sporting CP,2.417061,2.388588,3.590871,0.907333,0.092667
31,1208754,140,La Liga,Spain,Regular Season - 29,Real Madrid,Leganes,2.542099,2.357141,5.068590,0.947313,0.052687
55,1213132,235,Premier League,Russia,Regular Season - 22,CSKA Moscow,Makhachkala,2.483679,2.483679,3.549132,0.984681,0.015319


Explainer



In [57]:
def ensemble_handler(wining,model_dict):
    final_pred=0
    for m in  wining:
        final_pred+= model_dict[m]
    
    return final_pred/len(models)

In [58]:
class ensemble_handler:
    def __init__(self,winning,model_dict):
        self.winning= winning
        self.model_dict=model_dict

    def __call__(self,data):
        final_pred=0
        for m in  self.winning:
            final_pred+= self.model_dict[m].predict(X=data)
    
        return final_pred/len(models)

In [59]:
ens_c= ensemble_handler(wining,fitted_models)

In [60]:
explainer = shap.KernelExplainer(ens_c,test[list(input_cols)])

c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Using 2542 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [61]:
test_data=test[list(input_cols)]

In [105]:
for i in range(3):
    print(f"🔍 Explaining row {i}")
    shap_values = explainer.shap_values(test_data.iloc[i:i+1])

    exp_cols=pd.DataFrame(data={"Contribution":shap_values[0],
                                "Actual_Value":test_data.iloc[i]},index=input_cols)
    print(exp_cols[exp_cols.Contribution!=0].sort_values("Contribution",ascending=False))
    

🔍 Explaining row 0


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
100%|██████████| 1/1 [00:06<00:00,  6.79s/it]


                         Contribution  Actual_Value
Home                         0.042757         1.000
Shots_on_Goal                0.014075         5.000
Total_Shot_Taken_Oppose      0.012671         5.500
Shots_off_Goal               0.008477         6.500
Rank_Name_title             -0.006412         0.000
Ball_Possession_Rate        -0.008431         0.475
Rank_Name_Middle            -0.008490         0.000
Rank_Name_first_4_6         -0.008986         0.000
Rank_Name_last              -0.011821         1.000
Total_Passes                -0.016023       357.500
🔍 Explaining row 1


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
100%|██████████| 1/1 [00:06<00:00,  6.57s/it]


                             Contribution  Actual_Value
Rank_Name_last_Oppose            0.043825         1.000
Rank_Name_first_4_6              0.019108         1.000
Ball_Possession_Rate_Oppose      0.007512         0.475
Rank_Name_first_2_3             -0.002369         0.000
Corner_Kicks                    -0.003197         2.000
Ball_Possession_Rate            -0.005971         0.465
Total_Shot_Taken_Oppose         -0.012479         3.500
Shots_on_Goal                   -0.014855         2.500
Total_Passes                    -0.017530       378.500
Home                            -0.038810         0.000
🔍 Explaining row 2


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Arda\Downloads\Upwork\Portfolio\Betting_Asistant\predict_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
100%|██████████| 1/1 [00:06<00:00,  6.55s/it]

                             Contribution  Actual_Value
Home                             0.043183         1.000
Shots_on_Goal                    0.031710         5.500
Rank_Name_last_Oppose           -0.004011         0.000
Rank_Name_Middle                -0.004064         0.000
Rank_Name_first_4_6             -0.005321         0.000
Rank_Name_first_2_3             -0.005962         0.000
Ball_Possession_Rate_Oppose     -0.008739         0.605
Inside_Box_Shot_Rate            -0.008846         0.460
Total_Passes                    -0.012764       349.500
Rank_Name_first_2_3_Oppose      -0.026770         1.000


In [87]:
explainer.expected_value

0.38278029164517174

Home                           0.043505
Shots_on_Goal                  0.031748
Rank_Name_Middle_Oppose       -0.004909
Total_Shot_Taken_Oppose       -0.005284
Rank_Name_first_2_3           -0.005356
Rank_Name_first_4_6           -0.005429
Ball_Possession_Rate_Oppose   -0.008167
Inside_Box_Shot_Rate          -0.008866
Total_Passes                  -0.012728
Rank_Name_first_2_3_Oppose    -0.026101
dtype: float64

In [94]:
test_data

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_last_2_3_Oppose,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose
123417,1,5.0,6.5,14.0,4.0,2.0,0.475,5.5,0.5,357.5,...,0,0,0,0,1,0,0,0,0,1
123418,0,2.5,5.5,14.5,2.0,0.5,0.465,3.5,0.0,378.5,...,0,0,1,0,0,0,0,0,1,0
123419,1,5.5,2.0,14.0,5.5,2.0,0.535,2.0,0.0,349.5,...,0,0,1,0,0,0,0,0,0,1
123420,0,3.5,7.0,11.5,5.0,1.0,0.605,1.5,0.0,457.5,...,1,0,1,0,0,0,0,0,0,1
123421,1,3.5,4.5,10.5,3.5,1.0,0.540,2.0,0.0,428.5,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1,2.0,3.5,16.5,4.0,1.5,0.395,2.0,0.0,306.5,...,0,0,0,1,0,0,0,0,0,1
132757,0,5.5,5.5,11.0,4.5,1.0,0.545,1.0,0.0,417.0,...,0,0,1,0,0,0,0,0,0,1
132758,1,4.5,6.5,10.5,6.5,1.0,0.580,2.5,0.0,491.5,...,0,0,1,0,0,0,1,0,0,0
132759,1,4.5,6.0,12.5,6.0,4.0,0.595,3.5,0.0,472.5,...,0,0,0,0,1,0,0,0,1,0


In [93]:
test_data[input_cols].iloc[i:i+1]

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_last_2_3_Oppose,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose
123419,1,5.5,2.0,14.0,5.5,2.0,0.535,2.0,0.0,349.5,...,0,0,1,0,0,0,0,0,0,1


In [89]:
ens_c(test_data[input_cols])

array([0.40059617, 0.35801354, 0.38119541, ..., 0.47491036, 0.63863644,
       0.28775971], shape=(2542,))